In [29]:
pip install pillow


In [30]:
import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 
import cv2 
from PIL import Image, ImageOps, ImageFilter
import sys
from pathlib import Path

# 실행전 확인

- 원본이미지의 폴더 path
- 필터링적용한 이미지가 저장될 폴더 path

- 위의 두 폴더 path확인해  `save_filtered_images` 함수에 넣어줘야 함

## 함수 정의

- 이미지 파일이 있는 폴더에서 이미지를 하나하나 가져와 필터를 적용하는 함수임

In [31]:
def save_filtered_images(directory, filter_func, output_directory):
    # 출력 디렉토리가 없는 경우 생성
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # 디렉토리 내의 파일들을 모두 가져옴
    files = os.listdir(directory)

    for file in files:
        # 파일의 확장자 확인
        _, extension = os.path.splitext(file)
        if extension.lower() in ['.jpg', '.jpeg', '.png', '.gif']:
            # 이미지 파일인 경우에만 필터링을 적용
            image_path = os.path.join(directory, file)
            image = Image.open(image_path)
            filtered_image = filter_func(image)
            # 저장할 파일 경로와 이름 생성
            file_name = f"{filter_func.__name__}_{file}"  # 새로운 이름 설정
            output_path = os.path.join(output_directory, file_name)
            filtered_image.save(output_path)

# 이미지 필터링을 적용할 디렉토리 경로
directory_path = r'C:/dev/github/explore/webcam/tv'
# 필터링된 이미지를 저장할 디렉토리 경로
output_directory_path = r'C:/dev/github/explore/webcam/transform_tv'

## 필터정의

In [32]:
# roboflow 에서 (640,640)으로 일괄조정 예정 : 추후 사이즈 문제 발생시 알아서 적용하기
# def resize_image(image):
#     size = (256, 256)   ### 사이즈 지정
#     resized_image = image.resize(size)
#     return resized_image

rotation 

- 각도값 다양하게 변경해 적용가능

In [33]:
# Rotation : 각도 변경해서 사용가능
def rotate_image(image):
    rotated_image = image.rotate(45, expand=True)   ### 각도
    return rotated_image

def rotate_image_90(image):
    rotate_image_90 = image.rotate(90, expand=True)  
    return rotate_image_90

def rotate_image_270(image):
    rotate_image_270 = image.rotate(270, expand=True)  
    return rotate_image_270

# save_filtered_images(directory_path, rotate_image, output_directory_path)
# save_filtered_images(directory_path, rotate_image_90, output_directory_path)
# save_filtered_images(directory_path, rotate_image_270, output_directory_path)

예시 확인코드

In [34]:
# 예시 사진으로 결과물 확인가능
# image = r'C:\dev\github\.venv\03_final_pjt\.venv\preprocessing\data\00001496_004.jpg'  # 대상 이미지
# image = Image.open(image)
# rotate_image(image)

blurring

- kernal size 조정해 사용가능
- gaussizan의 경우는 홀수값만 적용가능

In [35]:
# blurring - blur filter 
def blur_filter(image):
    blur_filter = image.filter(ImageFilter.BLUR)
    return blur_filter

# blurring - Kernal size = 10
def blur_kernal(image):
    image_array = np.array(image)
    blur_kernel = cv2.blur(image_array, (10, 10))
    blur_kernel = Image.fromarray(blur_kernel)
    return blur_kernel

# blurring - blur_gaussian = 21
def blur_gaussian(image):
    image_array = np.array(image)
    blur_gaussian = cv2.medianBlur(image_array, 21)  ### 양의 홀수값만 사용가능
    blur_gaussian = Image.fromarray(blur_gaussian)
    return blur_gaussian

def smoothing(image):
    smoothing = image.filter(ImageFilter.SMOOTH)
    return smoothing

# save_filtered_images(directory_path, blur_filter, output_directory_path)
# save_filtered_images(directory_path, blur_kernal, output_directory_path)
# save_filtered_images(directory_path, blur_gaussian, output_directory_path)
# save_filtered_images(directory_path, smoothing, output_directory_path)

edge

In [36]:
# edge - fineedge 
def edge_fineedge(image):
    image = np.array(image)
    image_gray = Image.fromarray(image).convert('L')
    edge_fineedge = image_gray.filter(ImageFilter.FIND_EDGES)
    return edge_fineedge

# edge - smooth 
def edge_smooth(image):
    image = np.array(image)
    image_gray = Image.fromarray(image).convert('L')
    edge_smooth = image_gray.filter(ImageFilter.SMOOTH)
    return edge_smooth

# edge - enhance
def edge_enhance(image):
    image = np.array(image)
    image_gray = Image.fromarray(image.astype('uint8')).convert('L')
    # image_gray = Image.fromarray(image).convert('L')
    edge_enhance = image_gray.filter(ImageFilter.EDGE_ENHANCE)
    return edge_enhance

# edge - gaussian
def edge_gaussian(image):
    image = np.array(image)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edge_gaussian = cv2.GaussianBlur(image_gray, (5,5), 0)
    edge_gaussian = Image.fromarray(edge_gaussian)
    return edge_gaussian

# edge - canny
def edge_canny(image):
    image = np.array(image)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edge_canny = cv2.Canny(image_gray, 50, 300)
    edge_canny = Image.fromarray(edge_canny)
    return edge_canny

In [37]:
# save_filtered_images(directory_path, edge_fineedge, output_directory_path)
# save_filtered_images(directory_path, edge_smooth, output_directory_path)
# save_filtered_images(directory_path, edge_enhance, output_directory_path)
# save_filtered_images(directory_path, edge_gaussian, output_directory_path)
# save_filtered_images(directory_path, edge_canny, output_directory_path)

threshold

In [38]:
# adj_threshold = 100
def adj_threshold(image):
    image_gray = image.convert('L')
    threshold = 100
    adj_threshold = image_gray.point(lambda x: 255 if x > threshold else 0)
    return adj_threshold

def adj_threshold_150(image):
    image_gray = image.convert('L')
    threshold = 150
    adj_threshold_150 = image_gray.point(lambda x: 255 if x > threshold else 0)
    return adj_threshold_150

# thresholding - equalized
def threshold_equalized(image):
    image_gray = image.convert('L')
    threshold_value = 128
    image_thresholded = image_gray.point(lambda x: 255 if x >= threshold_value else 0)  # Thresholding
    image_filtered = image_thresholded.filter(ImageFilter.BLUR)  # Apply a blurring filter
    threshold_equalized = ImageOps.equalize(image_filtered)  # Apply histogram equalization
    return threshold_equalized

# thresholding - gaussian
def adaptiveThreshold_gaussian(image):
    image_gray = np.array(image.convert('L'))
    adaptiveThreshold_gaussian = cv2.adaptiveThreshold(image_gray,255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    adaptiveThreshold_gaussian = Image.fromarray(adaptiveThreshold_gaussian)
    return adaptiveThreshold_gaussian

# thresholding - mean
def adaptiveThreshold_mean(image):
    image_gray = np.array(image.convert('L'))
    adaptiveThreshold_mean = cv2.adaptiveThreshold(image_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    adaptiveThreshold_mean = Image.fromarray(adaptiveThreshold_mean)
    return adaptiveThreshold_mean

In [39]:
# save_filtered_images(directory_path, adj_threshold, output_directory_path)
# save_filtered_images(directory_path, threshold_equalized, output_directory_path)
# save_filtered_images(directory_path, adaptiveThreshold_gaussian, output_directory_path)
# save_filtered_images(directory_path, adaptiveThreshold_mean, output_directory_path)

erosion, dilation
 - 반복횟수 iterations를 크게할수록 뭉개짐

In [40]:
# Erosion
def erosion(image):
    image_gray = np.array(image.convert('L'))
    kernel = np.ones((3,3), np.uint8)
    erosion = cv2.erode(image_gray, kernel, iterations=1) # 반복횟수 iterations=1 (많이하면 뭉개짐)
    erosion = Image.fromarray(erosion)
    return erosion

# Dilation
def diilation(image):
    image_gray = np.array(image.convert('L'))
    kernel = np.ones((3,3), np.uint8)
    dilation = cv2.dilate(image_gray, kernel, iterations=3)
    dilation = Image.fromarray(dilation)
    return dilation

In [41]:
# save_filtered_images(directory_path, erosion, output_directory_path)
# save_filtered_images(directory_path, diilation, output_directory_path)

## 실행

 - 필터당 한개의 실행함수가 있음
 - 아래 실행시 대상폴더에 필터링이 적용된 파일들이 저장됨

In [42]:
# rotation
save_filtered_images(directory_path, rotate_image, output_directory_path)
# blurring
save_filtered_images(directory_path, blur_filter, output_directory_path)
save_filtered_images(directory_path, blur_kernal, output_directory_path)
save_filtered_images(directory_path, blur_gaussian, output_directory_path)
save_filtered_images(directory_path, smoothing, output_directory_path)
# edge
save_filtered_images(directory_path, edge_fineedge, output_directory_path)
save_filtered_images(directory_path, edge_smooth, output_directory_path)
save_filtered_images(directory_path, edge_enhance, output_directory_path)
save_filtered_images(directory_path, edge_gaussian, output_directory_path)
save_filtered_images(directory_path, edge_canny, output_directory_path)
# threshold
save_filtered_images(directory_path, adj_threshold, output_directory_path)
save_filtered_images(directory_path, threshold_equalized, output_directory_path)
save_filtered_images(directory_path, adaptiveThreshold_gaussian, output_directory_path)
save_filtered_images(directory_path, adaptiveThreshold_mean, output_directory_path)
# erosion, dilation
save_filtered_images(directory_path, erosion, output_directory_path)
save_filtered_images(directory_path, diilation, output_directory_path)